In [ ]:
from haystack.nodes import PDFToTextConverter
from haystack.nodes import PreProcessor
from pathlib import Path
from haystack.pipelines import Pipeline
import pandas as pd
import os

In [ ]:
df = pd.read_pickle('diw.pkl')

In [ ]:
meta_data = df.to_dict('records')
meta_data[0]

In [ ]:
converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["de","en"]
)

processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0
)


In [53]:
doc1 = converter.convert(file_path=Path("pdfs/Brounstein_2021.pdf"), meta=meta_data[0])
doc2 = converter.convert(file_path=Path("pdfs/Chen_2019.pdf"), meta=meta_data[1])


In [54]:
doc1[0].meta


{'title': 'Tuning the 3d printability and thermomechanical properties of radiation shields',
 'author': 'Brounstein, Zachary',
 'year': '2021',
 'abstract': 'Additive manufacturing, with its rapid advances in materials science, allows for researchers and companies to have the ability to create novel formulations and final parts that would have been difficult or near impossible to fabricate with traditional manufacturing methods. One such 3D printing technology, direct ink writing, is especially advantageous in fields requiring customizable parts with high amounts of functional fillers. Nuclear technology is a prime example of a field that necessitates new material design with regard to unique parts that also provide radiation shielding. Indeed, much effort has been focused on developing new rigid radiation shielding components, but DIW remains a less explored technology with a lot of potential for nuclear applications. In this study, DIW formulations that can behave as radiation shield

In [58]:
for count, filename in enumerate(os.listdir("pdfs/")):
    if count < 1:
        print(filename)
        doc = converter.convert(file_path=Path("pdfs/"+str(filename)), meta=meta_data[count])
        processed_docs = processor.process(doc)
        # cast to document store here
        # OR
        # for doc_count in len(processed_docs):
            # send processed_docs[doc_count].content to text-ada-002
            #processed_docs[doc_count].embedding = embedding

    else:
        break


Brounstein_2021.pdf
Chen_2019.pdf


In [62]:
doc[0].content

"Rheological behavior of titania ink and mechanical properties of\ntitania ceramic structures by 3D direct ink writing using high solid\nloading titania ceramic ink\nTao Chen a, b, Aihua Sun b, *, Chengyi Chu b, Huansheng Wu b, Jiang Wang a, Jianye Wang b,\nZhixiang Li b, Jianjun Guo b, Gaojie Xu b\na School of Materials Science and Engineering, Shanghai University, Baoshan District, Shanghai, 200444, PR China\nb Key Laboratory of Additive Manufacturing Materials of Zhejiang Province, Ningbo Institute of Materials Technology and Engineering, Chinese Academy of\nScience, Ningbo, 315201, PR China\na r t i c l e i n f o\nArticle history:\nReceived in revised form\nAvailable online 29 December 2018\nKeywords:\nTitania ceramic\nHigh solid loading\nDirect ink writing\nRheological behavior\nCompressive strength\na b s t r a c t\nIn this paper, high solid loading titania ceramic ink containing sodium hexametaphosphate and sodium\nalginate had been successfully prepared at room temperature. Rhe